<a href="https://colab.research.google.com/github/KimJuHan/pytorch_tutorial/blob/master/cnn_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import argparse
import torch
from torch import nn, optim, cuda
from torchvision import datasets, transforms
from torch.autograd import Variable

batch_size = 64
device = 'cuda' if cuda.is_available() else 'cpu'
train_dataset = datasets.MNIST(root='./data',
                               train=True,
                               download=True,
                               transform=transforms.ToTensor())

test_dataset = datasets.MNIST(root='./data',
                              train=False,
                              transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           shuffle=True,
                                           batch_size=batch_size)

test_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          shuffle=False,
                                          batch_size=batch_size)

In [0]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
    self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
    self.mp = nn.MaxPool2d(2)
    self.fc = nn.Linear(320, 10)

  def forward(self, x):
    in_size = x.size(0)
    x = nn.functional.relu(self.mp(self.conv1(x)))
    x = nn.functional.relu(self.mp(self.conv2(x)))
    x = x.view(in_size, -1)
    x = self.fc(x)
    return nn.functional.log_softmax(x)

model = Net()
model.to(device)

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [10]:
def train(epoch):
  model.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = nn.functional.cross_entropy(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % 10 == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)] | Loss: {:.6f}'.format(
          epoch, batch_idx*len(data), len(train_loader.dataset),
          100. * batch_idx / len(train_loader), loss.item()
      ))

def test():
  model.eval()
  test_loss = 0
  correct = 0
  for data, target in test_loader:
    data, target = data.to(device), target.to(device)
    output = model(data)
    test_loss += nn.functional.nll_loss(output, target, reduction='sum').data
    pred = output.data.max(1, keepdim=True)[1]
    correct += pred.eq(target.data.view_as(pred)).cpu().sum()

  test_loss /= len(test_loader.dataset)
  print('\nTest set: Average Loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
      test_loss, correct, len(test_loader.dataset),
      100. * correct / len(test_loader.dataset)
  ))

for epoch in range(0, 10):
  train(epoch)
  test()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


Train Epoch: 0 [0/60000 (0%)] | Loss: 0.132132
Train Epoch: 0 [640/60000 (1%)] | Loss: 0.092129
Train Epoch: 0 [1280/60000 (2%)] | Loss: 0.256218
Train Epoch: 0 [1920/60000 (3%)] | Loss: 0.098551
Train Epoch: 0 [2560/60000 (4%)] | Loss: 0.074719
Train Epoch: 0 [3200/60000 (5%)] | Loss: 0.055770
Train Epoch: 0 [3840/60000 (6%)] | Loss: 0.036316
Train Epoch: 0 [4480/60000 (7%)] | Loss: 0.202959
Train Epoch: 0 [5120/60000 (9%)] | Loss: 0.317817
Train Epoch: 0 [5760/60000 (10%)] | Loss: 0.073762
Train Epoch: 0 [6400/60000 (11%)] | Loss: 0.154636
Train Epoch: 0 [7040/60000 (12%)] | Loss: 0.117162
Train Epoch: 0 [7680/60000 (13%)] | Loss: 0.155263
Train Epoch: 0 [8320/60000 (14%)] | Loss: 0.113134
Train Epoch: 0 [8960/60000 (15%)] | Loss: 0.044173
Train Epoch: 0 [9600/60000 (16%)] | Loss: 0.106464
Train Epoch: 0 [10240/60000 (17%)] | Loss: 0.177684
Train Epoch: 0 [10880/60000 (18%)] | Loss: 0.159044
Train Epoch: 0 [11520/60000 (19%)] | Loss: 0.128416
Train Epoch: 0 [12160/60000 (20%)] | Loss